In [1]:
import pandas as pd 
import numpy as np 

In [2]:
def _open_cleaned_file(file, folder):
    """Reads the cleaned file. Adds it to the output dataframe we are working with form here on .
    Returns the list of the leading nan values in the list. Nan_list may be empty

    Args:
        file (str): file name

    Returns:
        List: List over nan values
    """
    nan_list = []
    file_object = open(folder + file, "r")
    file_object.readline()
    for line in file_object:
        if line[:2] == "#*":
            df_output = pd.read_csv(file_object, delim_whitespace=True, header=0, index_col=[0, 1, 2])
            break
        nan_list.append(eval(line[2:]))

    file_object.close()
    return nan_list, df_output

In [3]:
def _check_nan(df_output, col):
    """Returns a list of all nan indexes in the dataframe.txt
    Redundant

    Returns:
        np.ndarray: index array all nan indexes
    """
    # If it finds nan values in the data set, it returns the location as a tuple of the nan values
    # Potential issues is long periods of NaN values
    nan_values = df_output[col].isnull() # == nan_ident                  # Creates a series
    nr_nan_values = nan_values.value_counts()                                   # Counts values in a truth array
    if True in nr_nan_values.index:                                             # Check if any value is true
        loc_nan = nan_values[nan_values == True].index.values                   # Creates a list of tuples of NaN locations
        return loc_nan                                                          # Returns above list
    return 0

In [4]:
def _add_hyd_year(df_output):
        """Adds the hydrological year to the dataframe as a new column and sets it as column number 1, index 0
        """
        def assign_wy(row):
            if row.name[1] > 8:
                return(int(row.name[0] + 1))
            else:
                return(int(row.name[0]))
        df_output['WY'] = df_output.apply(lambda x: assign_wy(x), axis=1)     # Adds column
        df_output = df_output[["WY", "discharge"]]                             # Sorts order
        return df_output

In [5]:
def _segement_dataframe_baseflow(nan_list, df_output):
    """Creates a new column in the dataframe including the baseflow seperated values

    Args:
        nan_list (List): List of tuples of nan placements, output of the self._congregate_nan_list(x)
    """
    # Start of loop
    start = 0
    reflect = 30
    length_of_nan = []
    for tup in nan_list:
        length_of_nan.append(tup[2])

        input_array = df_output["discharge"].iloc[start:tup[0]].to_numpy()        # creates the input array of valid length
        baseflow = _baseflow_seperation(input_array)                              # finds the baseflow seperation from said input array
        df_output["Qb"].iloc[start:tup[0]] = baseflow                             # Adds baseflow to the dataframe

        start = tup[1] + 1
    # End of loop
    # Handles the tail of the dataframe, this should always be valid, but this makes sure it is valid.
    # If not valid set end to nan values.
    tail = len(df_output["discharge"].iloc[start:]) > reflect
    if tail:                                                                      # Fixes the tail if it exists.
        input_array = df_output["discharge"].iloc[start:].to_numpy()              # Creates the final array and handles that
        baseflow = _baseflow_seperation(input_array)                              # finds the baseflow seperation from said input array
        df_output["Qb"].iloc[start:] = baseflow                                   # Adds baseflow to the dataframe
    else:
        df_output["Qb"].iloc[start:] = np.nan(len(df_output["Qb"].iloc[start:]), np.nan)
    return df_output

In [6]:
def _baseflow_seperation(q, alpha=0.925, reflect=30):
    """Calculates the baseflow for a set dataframe using the Lyne-Hollick method.
    This function is based on the implementation recommended by Tony Ladson (2013) and his accompanying R-implementation avaiable at
    - DOI:       10.7158/W12-028.2013.17.1.
    - Code:      https://github.com/TonyLadson/BaseflowSeparation_LyneHollick 
    - Blogpost:  https://tonyladson.wordpress.com/2013/10/01/a-standard-approach-to-baseflow-separation-using-the-lyne-and-hollick-filter/

    Date for implementation: 2022/02/23 
    Crossreferenced last:    2022/02/23

    Freedoms have been taken in how data is handled and worked with as a different langauge is used.
    The essence of the recommended implementation is followed, while not all functions may be working.
    Some unnecessary implementation is done to stay true to the original R-implementation used as base. 

    The input array is the array for which the baseflow needs to be calculated, sequencing and slicing happens prior to this. 

    Args:
        q (np.ndarray): Input streamflow data in the form of a numpy array
        alpha (float, optional): Digital filter parameter based on the local and/or literature values for this parameter. Defaults to 0.925.
        reflect (int, optional): Amount of values to reflect. Defaults to recommended value for daily streamflow data. Defaults to 30.

    Returns:
        Tuple(np.ndarray): baseflow values for the streamflow.
    """
    # TODO: Look for simplifications if there is no need to save values they can be removed.

    # Expecting q to be a vector from numpy
    if not isinstance(q, np.ndarray):
        print("q must be a np.ndarray, it is ", type(q))
#         exit()

    if reflect >= len(q):
        print("Data set must be longer than the reflect period. Exiting...")
#         exit()

    if alpha < 0 or alpha >= 1:
        print("alpha must be between 0 and 1. Exiting...")
#         exit()

    def first_pass(q, a) -> pd.DataFrame:
        """Firsts forward pass of the dataframe, needs to be handled slightly differently than the later forward pass, due to being implemented as a dataframe.
        Choice of this was to avoid indexing issues when implementing.

        Args:
            q (np.ndarray): streamflow values
            a (float): digital filter parameter

        Returns:
            pd.DataFrame: Dataframe with columns ["qf", "qb"], which are the seperated quickflow and baseflow for use in the following passes
        """
        b = 0.5 * (1 + a)
        qf = np.zeros(len(q))  # Empty quickflow
        qf[0] = q[0]
        for i in range(1, len(qf)):
            qf[i] = a * qf[i - 1] + b * (q[i] - q[i - 1])

        qb1 = np.where(qf > 0, q - qf, q)

        return pd.DataFrame({"qf": qf, "qb": qb1})

    def backwards_pass(q, a) -> pd.DataFrame:
        """Backwards pass in the Lyne-Hollick method

        Args:
            q (pd.DataFrame): Dataframe with columns ["qf", "qb"], which are the seperated quickflow and baseflow for use in the following passes
            a (float): digital filter parameter

        Returns:
            pd.DataFrame: Dataframe with columns ["qf", "qb"], which are the seperated quickflow and baseflow for use in the following passes
        """
        n = len(q["qb"])
        qb = q["qb"]
        b = 0.5 * (1 + a)

        qf = np.zeros(n)  # Empty array
        qf[-1] = qb.iloc[-1]

        for i in range(n - 2, 0, -1):
            qf[i] = a * qf[i + 1] + b * (qb.iloc[i] - qb.iloc[i + 1])

        qb2 = np.where(qf > 0, qb - qf, qb)

        return pd.DataFrame({"qf": qf, "qb": qb2})

    def forward_pass(q, a) -> pd.DataFrame:
        """Forward pass, similar to the first pass, but uses dataframes instead

        Args:
            q (pd.DataFrame): Dataframe with columns ["qf", "qb"], which are the seperated quickflow and baseflow for use in the following passes
            a (float): digital filter parameter

        Returns:
            pd.DataFrame: Dataframe with columns ["qf", "qb"], which are the seperated quickflow and baseflow for use in the following passes
        """
        n = len(q["qb"])
        qb = q["qb"]
        b = 0.5 * (1 + a)

        qf = np.zeros(n)  # Empty array
        qf[0] = qb.iloc[0]

        for i in range(1, n):
            qf[i] = a * qf[i - 1] + b * (qb.iloc[i] - qb.iloc[i - 1])

        qb2 = np.where(qf > 0, qb - qf, qb)

        return pd.DataFrame({"qf": qf, "qb": qb2})

    q_in = np.pad(q, (reflect, reflect), mode="reflect")  # Pad the dataset

    # First pass always needed
    df_tmp = first_pass(q_in, alpha)

    df_tmp = backwards_pass(df_tmp, alpha)

    df_tmp = forward_pass(df_tmp, alpha)

    qb = df_tmp["qb"][reflect:-reflect].to_numpy()

    qb[qb < 0] = 0                  # Set values less than zero to zero

    return qb

In [7]:
def _nanindex(df_output, col, nan_ident=-9999):
    """Works on the current dataframe
    Retunrs as follows: # Start_Nan End_Nan Length_Nan Length_NoNan
    Start_Nan is the index value of the first nan values encountered
    End_Nan is the first non nan value after
    Length_Nan is the length of the sequence of Nan values
    Length_NoNan is the prior length of no nan values

    Returns:
        List[Tuple]: Tuple of the above property
    """
    data = df_output[col].to_numpy()
    l = []
    i = 0
    vq = 0
    
    while i < len(data):
        val = data[i]
        if val == nan_ident:
            v1 = i
            for j, val in enumerate(data[i:]):
                if val != nan_ident:
                    v2 = i + j
                    break
            l.append((v1, v2, v2 - v1, v1 - vq))
            vq = v2

            i += j
        i += 1
    return l

In [8]:
def _congregate_nan_list(nan_list):
    """Congregates the nan_lists.
    In most cases this functions does nothing.
    If Length_NoNan is shorter than reflect then it will congregate the sequence and essentially treat the data even though present as missing.
    So if Length_NoNan < reflect then we ignore the entire non nan segment and following calculations.

    Potential issue with this is if there is a single nan followed by n<reflect non-nans with a following nan. Where an entire segment is thrown when not needed.
    To counter his an interpolation may be used on the single nan or few nans to have a better dataset.

    Args:
        nan_list (List): List of nan values generated earlier by the self._nanindex function

    Returns:
        List: New list of nan tuples that are congregated.
    """
    # Here add handling of nan-values
    # Start_Nan End_Nan Length_Nan Length_NoNan
    # TODO: Make into function
    # TODO: Cleans up the nan_indicies. Do not want to do this before clean so I can manually check files.
    # TODO: Used to create merged nan_indicies
    if len(nan_list) == 0:
        return [], nan_list
    
    weights = []            # used in the weighting of the function, should contain length of data series
    reflect = 30

    start_index = []
    prev_nonan = []
    new_nan_list = []
    
    sequence = False
    
    
    for nan_indicies in nan_list:
        no_nan = nan_indicies[3]
        
        if sequence and no_nan > reflect:
#             print("New tuple - 1")
            new_tuple = (start_index[0], start_index[-1], start_index[-1] - start_index[0], prev_nonan[0])
            new_nan_list.append(new_tuple)
            
            start_index = []
            prev_nonan = []
            
            sequence = False
#         print(nan_indicies)
        start_index.append(nan_indicies[0])
        start_index.append(nan_indicies[1])
        
        prev_nonan.append(no_nan)

        if no_nan < reflect:
            continue
        else: 
            sequence = True
        if nan_indicies == nan_list[-1]:
#             print("New tuple - 2")
            new_tuple = (start_index[0], start_index[-1], start_index[-1] - start_index[0], prev_nonan[0])
            new_nan_list.append(new_tuple)
    return weights, new_nan_list


In [30]:
folder = "../GEO3000/code/data/discharge_data_100_cleaned/"
files = ["2.11.q", "2.265.q", "2.268.q", "2.279.q", "2.479.q", "2.633.q"]
for file in files:
    print(file)
    nan_list, df_output = _open_cleaned_file(file, folder)
    
    df_output = _add_hyd_year(df_output)
    
    df_output["Qb"] = pd.Series(dtype=np.float64)              # Adds a empty column for the baseflow
    
    weights, nan_list = _congregate_nan_list(nan_list)
      
    df_output = _segement_dataframe_baseflow(nan_list, df_output)                     # Segments the dataframe and does the baseflow seperation
    print(nan_list)
#     print(_check_nan(df_output, "Qb", np.nan))
    if file == "2.633.q":
        df_test = df_output.copy()

    df_output["nn"] = df_output["Qb"].isnull()
    df_grouped_null = df_output.groupby("WY")["nn"].sum()
#     display(df_grouped_null)

2.11.q


/home/sigurd/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


[]
2.265.q
[(9253, 10349, 1096, 9253)]
2.268.q
[(9619, 9932, 313, 9619)]
2.279.q
[]
2.479.q
[(122, 1066, 944, 122), (1135, 2516, 1381, 69), (2595, 2814, 219, 79)]
2.633.q
[(6698, 6699, 1, 6698), (6919, 6927, 8, 220), (7022, 7023, 1, 95)]


In [31]:
df_tmp = df_test.copy()


df_test = df_test.groupby("WY").sum()
df_test["BFI"] = df_test["Qb"]/df_test["discharge"]

def _pad_year(x):
    """
    Takes the array of the baseflow and returns the indexes of starting and ending nan values
    """
    x = np.hstack([ [False], x, [False] ])  # padding
    d = np.diff(x.astype(int))
    starts = np.where(d == 1)[0]
    ends = np.where(d == -1)[0]
    return starts, ends

def _weighted_avg(x, y, s, e):
    """
    Computes the weighted avg of the baseflow based on the starts and ends of the nan vlaues
    """
    n1 = 0
    n2 = 0
    w = []
    calc = []
    for i,j in zip(starts, ends):
        w.append(i - n1)
        calc.append(sum(x[n1:i])/sum(y[n1:i]))
        n1 = j
        
    qb = 0    
    for i in range(len(w)):
        qb += calc[i]*w[i]
    bfi = qb/sum(w)
    return bfi

# Find nan locations in the entire series of the baseflow values
df_tmp["nn"] = df_tmp["Qb"].isnull()

# Group the baseflow nans so it is per years
df_grouped = df_tmp.groupby("WY")["nn"].sum()
# Find all years where more than 5% of the baseflow values are missing. Change 19 if more values are acceptable
reqd_index_drop = df_grouped[df_grouped>=19].index.tolist()
# Find all years where there is more than 1 missing value
reqd_index = df_grouped[df_grouped > 0].index.tolist()

# reqd_index_drop is the years to drop regardless as too much data is missing
# reqd_index is the years where a weighted avg will be used. 

# Removes overlap between the two sets of years
for element in reqd_index_drop:
    if element in reqd_index:
        reqd_index.remove(element)
reqd_index = [1998, 1999]
# Loop over all years that are of interest
for year in reqd_index:
    print(year)
    # Extract that specific year
    tmp = df_tmp[df_tmp["WY"] == year]
    # Convert to numpy array
    tmp_n = tmp["nn"].to_numpy()
    # Computes the starts and ends of the nan values
    starts, ends = _pad_year(tmp_n)
    # Computes the weighted avg
    wa = _weighted_avg(tmp["Qb"], tmp["discharge"], starts, ends)
    df_test["BFI"].loc[year] = wa
df_test

1998
1999


,discharge,Qb,BFI
WY,,,
1980,472.744696,187.920541,0.397510
1981,587.515731,247.943745,0.422021
1982,534.418540,224.558832,0.420193
1983,642.928929,337.380699,0.524756
1984,437.202947,164.713263,0.376743
1985,722.678205,314.230058,0.434813
1986,667.985530,315.128265,0.471759
1987,671.874025,312.086318,0.464501
1988,1213.459365,642.836344,0.529755
